In [1]:
# add genes to algoithms without genes

import os
import numpy as np
import pandas as pd
import sys,os
import random
import copy

import subprocess

import matplotlib.pyplot as plt
import seaborn as sns

from utils.eval import find_best_matches, generate_exprs

from methods import NMF, PCA, sparse_PCA, moCluster, MOFA2, iClusterPlus

gene_sets_are_defined = ['NMF', 'sparse_PCA']


classifications={"Intrinsic":["Luminal","Basal","Her2","Normal","Claudin-low"],
                "SCMOD2":["ER-/HER2-","ER+/HER2- Low Prolif","ER+/HER2- High Prolif","HER2+"],
                "IHC":["IHC_TNBC","IHC_ER","IHC_HER2","IHC_PR"]}

file_metabric_annotation = '/local/DESMOND2_data/v6/preprocessed_v6/METABRIC_1904.annotation_v6.tsv'
file_metabric_expression = '/local/DESMOND2_data/v6/preprocessed_v6/METABRIC_1904_17Kgenes.log2_exprs_v6.tsv'
file_metabric_subtypes = '/local/DESMOND2_data/v6/preprocessed_v6/METABRIC_1904_17Kgenes.subtypes_and_signatures_v6.tsv'
file_tcga_annotation = '/local/DESMOND2_data/v6/preprocessed_v6/TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv'
file_tcga_expression = '/local/DESMOND2_data/v6/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_v6.tsv'
file_tcga_subtypes = '/local/DESMOND2_data/v6/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv'
file_gene_mapping = '/local/DESMOND2_data/v6/preprocessed_v6/gene_id_mapping.tsv'

out_dir = '/cosybio/project/hartung/unpast/unpast_real'

basename_t = "TCGA"
basename_m = "METABRIC" 
result_m = None
result_t = None

def add_genes(METHOD):
    global result_m, result_t
    
    method_name = METHOD.__name__.split('.')[-1]
    print('method_name:', method_name)

    #### Preparation
    # METABRIC
    file_path_m = file_metabric_expression
    output_path_m = os.path.join(out_dir, basename_m, method_name)
    ground_truth_file_m = file_metabric_annotation
    combinations_m = METHOD.generate_arg_list(file_path_m, output_path_m, ground_truth_file_m)
    # TCGA
    file_path_t = file_tcga_expression
    output_path_t = os.path.join(out_dir, basename_t, method_name)
    ground_truth_file_t = file_tcga_annotation
    combinations_t = METHOD.generate_arg_list(file_path_t, output_path_t, ground_truth_file_t)
    
    for _iteration, (comb_m, comb_t) in enumerate(zip(combinations_m, combinations_t)):
        
        if os.path.isfile(os.path.join(comb_m['output_path'], 'result.with_genes.tsv')) and os.path.isfile(os.path.join(comb_t['output_path'], 'result.with_genes.tsv')):
            print('Skipping because output files exist')
            continue
        
        result_m, _ = METHOD.run_real(comb_m, is_terminated=True)
        
        # save result as tsv for add genes input
        output_file = os.path.join(comb_m['output_path'], 'result.tsv')
        
        result_m['samples'] = result_m['samples'].map(list).map(lambda x: ' '.join(x))
        result_m.to_csv(output_file, sep='\t')
        
        print('output_file', output_file)

        process_m = subprocess.Popen([f"Rscript", "/home/bba1401/Projects/unpast/DESMOND2/utils/add_genes.R", output_file, file_metabric_expression, "FALSE"])
        
        
        
        result_t, _ = METHOD.run_real(comb_t, is_terminated=True)
        
        # save result as tsv for add genes input
        output_file = os.path.join(comb_t['output_path'], 'result.tsv')
        
        result_t['samples'] = result_t['samples'].map(list).map(lambda x: ' '.join(x))
        
        result_t.to_csv(output_file, sep='\t')
        print('output_file', output_file)

        process_t = subprocess.Popen([f"Rscript", "/home/bba1401/Projects/unpast/DESMOND2/utils/add_genes.R", output_file, file_tcga_expression, "TRUE"])
        
        process_m.wait()
        process_t.wait()
        


In [2]:
method = moCluster
add_genes(method)

method_name: moCluster
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipping because output files exist
Skipp

KeyboardInterrupt: 

In [3]:
add_genes(iClusterPlus)

method_name: iClusterPlus
output_file /cosybio/project/hartung/unpast/unpast_real/METABRIC/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=5/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.tsv
output_file /cosybio/project/hartung/unpast/unpast_real/TCGA/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=5/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.tsv
/cosybio/project/hartung/unpast/unpast_real/TCGA/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=5/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.with_genes.tsv 
/cosybio/project/hartung/unpast/unpast_real/TCGA/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=5/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.with_genes.tsv 
/cosybio/project/hartung/unpast/unpast_real/METABRIC/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_

KeyboardInterrupt: 

/cosybio/project/hartung/unpast/unpast_real/TCGA/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=10/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.with_genes.tsv 


In [4]:
add_genes(MOFA2)

method_name: MOFA2
output_file /cosybio/project/hartung/unpast/unpast_real/METABRIC/MOFA2/n_factors=1/n_cluster=2/random_state=1/ard_weights=True/ard_factors=False/likelihood=gaussian/spikeslab_weights=True/spikeslab_factors=False/result.tsv
output_file /cosybio/project/hartung/unpast/unpast_real/TCGA/MOFA2/n_factors=1/n_cluster=2/random_state=1/ard_weights=True/ard_factors=False/likelihood=gaussian/spikeslab_weights=True/spikeslab_factors=False/result.tsv
/cosybio/project/hartung/unpast/unpast_real/TCGA/MOFA2/n_factors=1/n_cluster=2/random_state=1/ard_weights=True/ard_factors=False/likelihood=gaussian/spikeslab_weights=True/spikeslab_factors=False/result.with_genes.tsv 
/cosybio/project/hartung/unpast/unpast_real/METABRIC/iClusterPlus/random_state=1/n_cluster=2/type=gaussian/n_burnin=200/n_draw=200/lambda_n=10/maxiter=20/sdev=0.05/eps=0.0001/lambda_scale=1/result.with_genes.tsv 


KeyboardInterrupt: 

/cosybio/project/hartung/unpast/unpast_real/METABRIC/MOFA2/n_factors=1/n_cluster=2/random_state=1/ard_weights=True/ard_factors=False/likelihood=gaussian/spikeslab_weights=True/spikeslab_factors=False/result.with_genes.tsv 


In [ ]:
result_m

,samples,n_samples
1,"[MB-5162, MB-5393, MB-6217, MB-4591, MB-4046, ...",457
2,"[MB-6305, MB-0501, MB-3123, MB-0644, MB-5562, ...",1447


In [ ]:
result_t

,samples,n_samples
1,"[TCGA-EW-A1J5-01, TCGA-A2-A0YJ-01, TCGA-LL-A73...",712
2,"[TCGA-OL-A5RX-01, TCGA-A2-A0CS-01, TCGA-OL-A66...",367


Coefficients not estimable: bic 


Error in contrasts.fit(fit, contrasts_matrix) : 
  trying to take contrast of non-estimable coefficient
Calls: apply -> FUN -> find_DE_genes -> contrasts.fit
In addition: Warning message:
Partial NA coefficients for 17158 probe(s) 
Execution halted


In [ ]:
pd.read_csv('/cosybio/project/hartung/unpast/unpast_real/METABRIC/moCluster/n_dimensions=1/n_cluster=2/random_state=1/solver=fast/center=True/method=globalScore/option=lambda1/scale=False/k=0.1/result.tsv', sep='\t')

,samples,n_samples
0,MB-5393 MB-4591 MB-6305 MB-0501 MB-3123 MB-064...,1825
1,MB-4046 MB-6065 MB-4395 MB-4798 MB-4332 MB-483...,79


In [ ]:
pd.read_csv('/home/bba1401/Projects/unpast/DESMOND2/evaluation/factorization/METABRIC_1904_17Kgenes.Kmeans.clusters.tsv', sep='\t')

,Unnamed: 0,samples,n_samples
0,0,MB-0000 MB-0005 MB-0014 MB-0022 MB-0048 MB-005...,509
1,1,MB-0147 MB-3110 MB-3153 MB-3167 MB-4001 MB-400...,137
2,2,MB-0006 MB-0054 MB-0060 MB-0079 MB-0093 MB-010...,371
3,3,MB-0045 MB-0062 MB-0100 MB-0115 MB-0149 MB-015...,225
4,4,MB-0002 MB-0008 MB-0010 MB-0028 MB-0035 MB-003...,662


Error: Negative counts not allowed
Execution halted
